In [1]:
import pandas as pd
from collections import Counter
from datetime import datetime, timedelta
import os

In [2]:
df_custom=pd.read_csv('cs.csv')

In [3]:
df_news=pd.read_csv('predict_news.csv')

In [4]:
custom=df_custom

In [5]:
news=df_news

In [6]:
custom1=custom.iloc[:,[0,1,2]].dropna()
custom2=custom.iloc[:,[0,1,4]].dropna()

In [7]:
news1=news[news['sentiment']==1].iloc[:,[1,-2]]
news2=news[news['sentiment']==2].iloc[:,[1,-2]]

In [8]:
custom1=custom1[custom1['BUY'].isin(news1['단축코드'])]
custom2=custom2[custom2['SELL'].isin(news2['단축코드'])]
news1 = news1[news1['단축코드'].isin(custom1['BUY'])]
news2 = news2[news2['단축코드'].isin(custom2['SELL'])]

In [22]:
custom1_train=custom1[custom1['DATE']<20230301]
news1_train=news1[news1['DATE_TIME']<20230401]
custom1_test=custom1[custom1['DATE']>=20230301]
news1_test=news1[news1['DATE_TIME']>=20230401]
custom2_train=custom2[custom2['DATE']<20230301]
news2_train=news2[news2['DATE_TIME']<20230401]
custom2_test=custom2[custom2['DATE']>=20230301]
news2_test=news2[news2['DATE_TIME']>=20230401]

In [20]:
def convert_to_datetime(date_str):
    return pd.to_datetime(date_str, format='%Y%m%d')

In [23]:
custom1_train['DATE'] = custom1_train['DATE'].apply(convert_to_datetime)
custom2_train['DATE'] = custom2_train['DATE'].apply(convert_to_datetime)
custom1_test['DATE'] = custom1_test['DATE'].apply(convert_to_datetime)
custom2_test['DATE'] = custom2_test['DATE'].apply(convert_to_datetime)
news1_train['DATE_TIME'] = news1_train['DATE_TIME'].apply(convert_to_datetime)
news2_train['DATE_TIME'] = news2_train['DATE_TIME'].apply(convert_to_datetime)
news1_test['DATE_TIME'] = news1_test['DATE_TIME'].apply(convert_to_datetime)
news2_test['DATE_TIME'] = news2_test['DATE_TIME'].apply(convert_to_datetime)

C:\Users\horri\AppData\Local\Temp\ipykernel_25340\351439433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custom1_train['DATE'] = custom1_train['DATE'].apply(convert_to_datetime)
C:\Users\horri\AppData\Local\Temp\ipykernel_25340\351439433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  custom2_train['DATE'] = custom2_train['DATE'].apply(convert_to_datetime)
C:\Users\horri\AppData\Local\Temp\ipykernel_25340\351439433.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [24]:
custom1_train

,NUM,DATE,BUY
4,2,2023-02-10,302440
9,3,2022-12-10,170030
10,3,2022-12-20,207760
11,3,2022-12-31,419120
12,3,2023-01-10,054920
...,...,...,...
7097406,803308,2022-12-10,099430
7097407,803308,2022-12-31,099430
7097408,803308,2023-01-31,099430
7097409,803308,2023-02-10,099430


In [34]:
news1_train=news1_train.drop_duplicates()
news2_train=news2_train.drop_duplicates()
news1_test=news1_test.drop_duplicates()
news2_test=news2_test.drop_duplicates()

In [ ]:
customers1 = []
customers_in_one_month1=[]
for index, row in news1_train.iterrows():
    news1_date = row['DATE_TIME']
    term = news1_date - timedelta(days=1)
    customers_in_one_month1 = custom1_train[
        (custom1_train['BUY'] == row['단축코드']) &
        (custom1_train['DATE'] >= term) &
        (custom1_train['DATE'] < news1_date)
    ]['NUM'].tolist()
    customers1.extend(customers_in_one_month1)

In [ ]:
customers1

In [ ]:
customers2 = pd.DataFrame([])
customers_in_one_month2=[]
for index, row in news2_train.iterrows():
    news2_date = row['DATE_TIME']
    term = news2_date - timedelta(days=1)
    customers_in_one_month2 = custom2_train[
        (custom2_train['SELL'] == row['단축코드']) &
        (custom2_train['DATE'] >= term) &
        (custom2_train['DATE'] < news2_date)
    ]['NUM'].tolist()
    customers1.extend(customers_in_one_month1)

In [ ]:
customers2

In [ ]:
customers_buy=pd.DataFrame(customers1['NUM'].value_counts(sort=False))

In [ ]:
customers_sell=pd.DataFrame(customers2['NUM'].value_counts(sort=False))

In [ ]:
os.chdir('C:/Users/horri/Desktop/대외활동/비어플/미래에셋 공모전')

In [ ]:
customers_buy.to_csv('buy_1days_only.csv',index=False)

In [ ]:
customers_sell.to_csv('sell_1days_only.csv', index=False)